## prepare images

## Use three models to demonstrate a typical images composition use case
1. FOPAHeatMapModel can predict the rationality scores for all locations/scales given a background-foreground pair, and output the composite image with optimal location/scale.
2. ImageHarmonizationModel adjusts the foreground illumination to be compatible the background given photorealistic background and photorealistic foreground.
3. ShadowGenerationModel generates plausible shadow for the inserted object in a composite image.

In [ ]:
# create a sagemaker runtie client ready to inference
import boto3
import json

# Initialize SageMaker runtime client
client = boto3.client("sagemaker-runtime")

# Define the endpoint name
endpoint_name = 'libcom'

import json
from PIL import Image
from IPython.display import Image
from utils.process_image import *

#### Provide a foreground image, a background image, and a foreground mask to obtain a heatmap and an automatically composited image of the foreground and background. There are various methods to obtain the mask, such as SAM. Note that the foreground mask generated by SAM has black as the foreground, and it needs to be converted into a mask where the foreground part is white for further use.


In [ ]:
### If we obtaining a mask image from SAM, we need to invert the black and white pixels of the mask. In the mask used by Libcom, the foreground pixels are white.
import cv2
test_dir  = 'uploads/'
image=cv2.imread(test_dir+'comp_mask_black.jpg')
inverted_image=cv2.bitwise_not(image)
cv2.imwrite(test_dir+'comp_mask.png',inverted_image)

In [ ]:
## Generate FOPA image and placement suggestion
background_path = "uploads/bg.png"
foreground_path = "uploads/fg.jpg"
foreground_mask_path = "uploads/fg_mask.png"
bg=encode_image(background_path)
fg=encode_image(foreground_path)
fg_mask=encode_image(foreground_mask_path)

function = "heatmap_compose"

payload = {
    "function":function,
    "background":bg,
    "foreground":fg,
    "foreground_mask":fg_mask
}

# Prepare the request body as a JSON object
request_body = json.dumps(payload)


In [ ]:
# Send the inference request to the endpoint
response = client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=request_body,
    ContentType="application/json"
)

# Parse the response
response_body = response["Body"].read().decode("utf-8")
fopa_result = json.loads(response_body)

bbox=fopa_result['bboxes']
print(bbox)
heatmap=save_image_from_base64(fopa_result['heatmap_image'],'results','heatmap')
comp_img=save_image_from_base64(fopa_result['comp_image'],'results','heatmap')
comp_mask=save_image_from_base64(fopa_result['comp_mask'],'results','heatmap')
grid_img  = make_image_grid([heatmap, comp_img, comp_mask])

from IPython.display import display,Image

cv2.imwrite(heatmap, grid_img)
display(Image(filename=heatmap))


### Image Compose Request
you can also call compose function directly with on demand positions


In [ ]:
# prepare images
background_path = "uploads/bg.png"
foreground_path = "uploads/fg.jpg"
foreground_mask_path = "uploads/fg_mask.png"
bg=encode_image(background_path)
fg=encode_image(foreground_path)
fg_mask=encode_image(foreground_mask_path)

function = "simple_compose"

bbox = [376, 187, 550, 791]

print(bbox)
payload = {
    "function":function,
    "background":bg,
    "foreground":fg,
    "foreground_mask":fg_mask,
    "bbox":bbox
}
# Prepare the request body as a JSON object
request_body = json.dumps(payload)



In [ ]:
# prepare function to encode and decode images to process inference request and response

# Send Image Compose inference request to the endpoint
response = client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=request_body,
    ContentType="application/json"
)

# Parse the response
response_body = response["Body"].read().decode("utf-8")
compose_result = json.loads(response_body)

# Process the result
#decode_image(compose_result['comp_image'])
#decode_image(compose_result['comp_mask'])

comp_image=save_image_from_base64(compose_result['comp_image'],'results','comp_image')
grid_img  = make_image_grid([foreground_path, background_path, comp_image])

from IPython.display import display,Image

cv2.imwrite(comp_image, grid_img)
display(Image(filename=comp_image))


## Image Harmonization using PCTNet and CDTNet

In [ ]:
# choose cdt mode

comp_image_path = "uploads/comp_image.png"
comp_mask_path = "uploads/comp_mask.png"
comp_image=encode_image(comp_image_path)
comp_mask=encode_image(comp_mask_path)

function = "cdt_mode"

payload = {
    "function":function,
    "comp_image":comp_image,
    "comp_mask":comp_mask
}

#payload
# Prepare the request body as a JSON object
request_body = json.dumps(payload)


In [ ]:
# Send the inference request to the endpoint
response = client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=request_body,
    ContentType="application/json"
)

# Parse the response
response_body = response["Body"].read().decode("utf-8")
harmony_result = json.loads(response_body)

# Process the result
#decode_image(harmony_result['pct_result'])

harmony_image=save_image_from_base64(harmony_result['cdt_result'],'results','harmony')
grid_img  = make_image_grid([comp_image_path, harmony_image])

from IPython.display import display,Image

cv2.imwrite(harmony_image, grid_img)
display(Image(filename=harmony_image))

In [ ]:
# choose pct mode

comp_image_path = "uploads/comp_image.png"
comp_mask_path = "uploads/comp_mask.png"
comp_image=encode_image(comp_image_path)
comp_mask=encode_image(comp_mask_path)

function = "pct_mode"

payload = {
    "function":function,
    "comp_image":comp_image,
    "comp_mask":comp_mask
}

#payload
# Prepare the request body as a JSON object
request_body = json.dumps(payload)


In [ ]:
# Send the inference request to the endpoint
response = client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=request_body,
    ContentType="application/json"
)

# Parse the response
response_body = response["Body"].read().decode("utf-8")
harmony_result = json.loads(response_body)

# Process the result
#print("Inference result:", harmony_result)
#decode_image(harmony_result['pct_result'])

harmony_image=save_image_from_base64(harmony_result['pct_result'],'results','harmony')
grid_img  = make_image_grid([comp_image_path, harmony_image])

from IPython.display import display,Image

cv2.imwrite(harmony_image, grid_img)
display(Image(filename=harmony_image))


## Shadow Generation

In [ ]:
# generate shadow
comp_img = "uploads/comp_image.png"
comp_mask = "uploads/comp_mask.png"
comp_img_b64=encode_image(comp_img)
comp_mask_b64=encode_image(comp_mask)

function = "gn_shadow"
number = 1

payload = {
    "function":function,
    "comp_image":comp_img_b64,
    "comp_mask":comp_mask_b64,
    "number":number
}
#payload

# Prepare the request body as a JSON object
request_body = json.dumps(payload)


In [ ]:
# Send the inference request to the endpoint
response = client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=request_body,
    ContentType="application/json"
)

# Parse the response
response_body = response["Body"].read().decode("utf-8")
shadow_result = json.loads(response_body)

# Process the result
#print("Inference result:", shadow_result)

shadow_image=save_image_from_base64(shadow_result['shadow'],'results','shadow')
grid_img  = make_image_grid([comp_image_path, shadow_image])

from IPython.display import display,Image

cv2.imwrite(shadow_image, grid_img)
display(Image(filename=shadow_image))